<img src="http://nci.org.au/wp-content/themes/nci/img/img-logo-large.png", width=400>

# Analysing and visualising data from multiple sources in QGIS


### In this notebook:
- How to use **QGIS** (http://qgis.org) as a tool for interacting with data using web data services and on the VDI filesystem.
- Obtaining data from a Web Coverage Service (WCS)
- Obtaining data from a web-based vector data service
- Obtaining data from the NCI file system using the NetCDF Operators (NCO)
- Using QGIS to fuse data and create a shareable, interactive 3D map rendered using webGL in a browser 

We are going to make an interactive version of the map below, starting with data from three different sources merged using QGIS. We will also see some key differences between data access methods.

![End result](./qgis.images/0-3d.map.png "End result of this session")

#### The following material uses Geoscience Australia's Elevation Data Collection which is available under the Create Commons License 4.0 through NCI's THREDDS Data Server. For more information on the collection and licensing, please [click here](https://geonetwork.nci.org.au/geonetwork/srv/eng/catalog.search#/metadata/f9082_1236_9859_8989). Also used is ANU Water and Landscape Dynamics tree cover product, available under the Create Commons License 4.0 through NCI's THREDDS Data Server. For more information on the collection and licensing, please [click here](http://geonetwork.nci.org.au/geonetwork/srv/eng/catalog.search#/metadata/f1104_2906_7276_3004).

## The program for this session is:

### Introduction and setup
1. <a href = "#introduction">QGIS - what and why?</a>
2. <a href = "#question">Define a research question</a>
3. <a href = "#datasets">Determine which datasets we can use to investigate our question</a>
4. <a href = "#servicechoice">Deciding on data manipulation strategies</a>


### Making your map
1. <a href = "#workspace">Create some working spaces</a>
2. <a href = "#ingestdata">Find and import web coverage raster data</a>
3. <a href = "#fs">Find and manipulate data on the filesystem</a>
4. <a href = "#qgis">Load QGIS (Quantum GIS, http://qgis.org) on the VDI</a>
5. <a href = "#loadraster">Import elevation and tree cover layers into QGIS</a>
6. <a href = "#rasterstyle">Styling raster data for visualisation</a>
7. <a href = "#ingestvector">Find cadastral data and import it into QGIS</a>
8. <a href = "#plugins">Install two useful QGIS plugins</a>
9. <a href = "#zonalstats">Analysis using zonal statistics</a>
10. <a href = "#visualise">Using a 3D visualisation to complete the picture</a>
11. <a href = "#interpret">Interpreting our results</a>
12. <a href = "#share">Share our new map</a>

# Introduction and setup

## <a id = "introduction"></a>1. QGIS - what and why?

### a. What is QGIS?

QGIS (Quantum GIS) is an application for geospatial data analysis and visualisation, focussed on 2D (raster and vector) datasets, and wrapped in a straightforward graphical user interface. It can also be used as a Python library, but for this session we use the graphical interface.

### b. I have [insert many other tools] on the VDI - why use QGIS?

QGIS can be used on the VDI as a rapid-prototyping tool. If you've got all the infrastructure you need available in your project space and don't need it, that's great!

We hope to demonstrate some ways which QGIS can help you quickly develop shareable, multi-source analyses quickly and simply.

### c. What  is the general approach of this exercise?

Imagine you've just run a model or synthesised some new analytical data and want to quickly compare your results with other data sources, inspect the results, and share with colleagues - essentially create a visualisation you can demonstrate.


## <a id = "question"></a>2. Define a question

For this session the question we want to address is:

#### *How are the 'hilliness' of suburban areas and and 'tree cover' in Canberra related?; and do hillier sections have more or less tree cover than flatter sections?*

## <a name = "datasets"></a>3. Decide on some useful datasets

To answer our question we need some data on topography, vegetation and where block boundaries are.

From NCI's collections we can use:
* **A Shuttle Radar Topography Mission raster DEM at 1 arc-second resolution**
* **Some tree cover indices from the ANU Water and Landscape Dynamics collection: /g/data2/ub8/**

NCI doesn't hold cadastral data, so we'll get those from the local land administrator:
* **ACT cadastral data outlining sections (held outside NCI)**

*Note - here, sections are collections of 50 or so individual house blocks. Our elevation and tree cover data are too coarse for individual house blocks - and the vector data file for blocks is massive!*

These data will come in three different formats:
1. GeoTIFF (constructed from a NetCDF file using WCS)
2. NetCDF4
3. Vector data as GeoJSON, which we will save as Geopackage **or** ESRI shapefile for use in QGIS (we will get these data in part 2 of this session)

## <a name = "servicechoice"></a>4. Deciding on data manipulation strategies

The use case for this exercise is to quickly merge prototype data (eg model output, analysis results) with relevant geospatial data for initial analysis and sharing. We will use three different approaches to obtain data for this map:

1. A Web Coverage Service (WCS) request - since we can find data via the NCI service catalogues more easily than on the file system
2. The file system - this is your own data, that may not be published yet.
3. A web protocol request for data that is not held at NCI.

The WCS request uses only a web browser. For filesystem data we use the NetCDF operators (**NCO**, http://nco.sourceforge.net/) to subset and manipulate our own data. And for external web service requests we will use **QGIS** native capabilities to ingest data.

<br />
<div class="alert alert-info">
<h3>Finding out more about the data we are using</h3>

<h3>Elevation</h3>
<p>
We use an SRTM elevation mosaic, held in Geoscience Australia's elevation reference collection:</p>
<ul>
<li><b>THREDDS:</b> http://dapds00.nci.org.au/thredds/catalog/rr1/Elevation/NetCDF/catalog.html</li>
<li><b>Geonetwork:</b> https://geonetwork.nci.org.au/geonetwork/srv/eng/catalog.search#/metadata/f9082_1236_9859_8989</li>
<li><b>Filesystem:</b> /g/data1/rr1/Elevation/NetCDF/</li>
</ul>
<p>
Licensed under Creative Commons (CCBY4): http://dapds00.nci.org.au/thredds/fileServer/licenses/rr1_licence.pdf</p>

<h3>Tree cover</h3>
<p>
These data come from the ANU Water and Landscape Dynamics collection:</p>
<ul>
<li><b>THREDDS:</b> http://dapds00.nci.org.au/thredds/catalog/ub8/au/catalog.html</li>
<li><b>Geonetwork:</b> http://geonetwork.nci.org.au/geonetwork/srv/eng/catalog.search#/metadata/f1104_2906_7276_3004</li>
<li><b>Filesystem:</b> /g/data1/ub8/au/treecover/</li>
</ul>
<p>
Licensed under Creative Commons (CCBY4): http://dapds00.nci.org.au/thredds/fileServer/licenses/ub8_licence.txt</p>

<h3>Cadastral data from ACTMAPi</h3>
<p>
Data used to define cadastral sections in the ACT are obtained from the ACT Government's ACTMAPi system (http://www.actmapi.act.gov.au). Data are released under a creative commons license (CCBY4): https://creativecommons.org/licenses/by/4.0/
<p>
The section data landing page is here: http://actmapi.actgov.opendata.arcgis.com/datasets/eedcda7873934e789093b093521b0299_3 ...where you can obtain the data via API links or direct download (for example, as a shapefile).</p>
<p>
A GeoJSON version of the data can be downloaded at this link: http://actmapi.actgov.opendata.arcgis.com/datasets/eedcda7873934e789093b093521b0299_3</p>


#### At this point you should have some idea about what the task is, and the data we are using. You should have a GeoTIFF elevation model and a NetCDF treecover dataset in ~/qgis_vis_data (or your equivalent). 

# Assembling your map using QGIS

## <a name = "workspace"></a>1. Create some working spaces

We need two directories to work with here - one to hold data and your QGIS project, one to hold the resulting visualisation.

```
$ mkdir ./qgis_vis_data
```
...to hold some data subsets we will create, and:

```
$ mkdir ./qgis_vis_map
```
...to hold our finished map. We will use these throughout the session.

## <a name = "ingestdata"></a>3. Get topography data using Web Coverage Services

For topography we need a terrain model in a raster format, e.g. GeoTIFF, which covers Canberra. We will collect a subset from the Shuttle Radar Topography Mission (SRTM) mosaic, held as a NetCDF file in Geoscience Australia's elevation collection (http://dapds00.nci.org.au/thredds/catalog/rr1/Elevation/catalog.html)

Clicking the link below will result in your browser asking to download a file named **WCSxxxxxxxxxx.tif**, where **xxxxxxxxx** is a series of numbers:

** http://dapds00.nci.org.au/thredds/wcs/rr1/Elevation/NetCDF/1secSRTM_DEMs_v1.0/DEM/Elevation_1secSRTM_DEMs_v1.0_DEM_Mosaic_dem1sv1_0.nc?service=WCS&version=1.0.0&request=GetCoverage&Coverage=elevation&bbox=148.7,-35.8,149.5,-35.1&format=GeoTIFF_Float **

Save the WCSxxxxxxxxxx.tif file, then move it from your ```Downloads``` directory it to your ```act_vis_data``` directory and rename it to something memorable, for example ```act_dem.tif```

What happened here? We used the THREDDS **web coverage service (WCS)** to clip a small chunk from the NetCDF elevation mosaic and give us data as a GeoTIFF.

A quick summary of how the request is constructed is below:

* the path to the data: http://dapds00.nci.org.au/thredds/wcs/rr1/Elevation/NetCDF/1secSRTM_DEMs_v1.0/DEM/
* the dataset: Elevation_1secSRTM_DEMs_v1.0_DEM_Mosaic_dem1sv1_0.nc
* the service: service=WCS
* the service version: version=1.0.0
* the thing we want to do (get a coverage): request=GetCoverage
* the coverage (or layer) we want to get: Coverage=elevation
* the boundary of the layer we want: bbox=148.7,-35.8,149.5,-35.1
* the format we want to get our coverage as: format=GeoTIFF_Float

*Note the use of GeoTIFF_Float - required for THREDDS to return a GeoTIFF with pixels holding data ranges. Other OWS providers (eg Geoserver) might not require the _Float suffix*

## <a name = "fs"></a>6. Acquire tree cover data from the file system

Here we pull some data from a netCDF file straight from the NCI file system - but we don't want all of Australia - let's stick to our region of interest, and use the NetCDF Operators (**NCO** - http://nco.sourceforge.net/) to grab the part we need.

In a **new** terminal move to your ```qgis_vis_data``` directory and load NCO:

```
$ cd ~/qgis_vis_data
$ module load nco/4.5.3
```

Use the NCO utility ```ncks``` to clip a region from the ANU WALD TreeCover dataset for 2015 from ```/g/data1/ub8/```:

```
$ ncks -v TreeCover -d latitude,-35.8,-35.1 -d longitude,148.7,149.5  /g/data1/ub8/au/treecover/ANUWALD.TreeCover.25m.2015.nc ./treecover_2015_-35.8-35.1_148.7_149.5.nc
```

QGIS will panic if you attempt to load your new netCDF subset, because it misinterprets the order of latitude and longitude. A quick call to another NCO utility fixes that - here we use ```ncpdq``` to swap the axis order in the TreeCover variable for our subset:

```
$ ncpdq -v TreeCover -a latitude,longitude treecover_2015_-35.8-35.1_148.7_149.5.nc treecover_2015_-35.8-35.1_148.7_149.5.nc
```

When asked about over-writing a file, choose **o** to overwrite. Now we have a small subset of the treecover dataset to work with.

<br />
<div class="alert alert-info">
<p>
You could just as easily load a complete NetCDF file from /g/data into QGIS - <i>as long as the underlying file format meets defined netCDF conventions</i>. Try adding this gravity map - it meets NetCDF-CF, and loads without any modification: /g/data/rr2/National_Coverages/onshore_Bouguer_offshore_Freeair_gravity_geodetic_June_2009/
onshore_Bouguer_offshore_Freeair_gravity_geodetic_June_2009.nc</p>
<p>
Here, we demonstrate a method of quickly viewing parts of your output data - even if they are not yet fully QC'ed and need a little massaging to get going.</p>
</div>

## <a name = "qgis"></a>4. Start QGIS on the VDI

Like most VDI applications, we use the ```module load``` system to acquire QGIS, GDAL and NCO. In your terminal window type:

```
module purge
module load proj/4.8.0 gdal/1.11.1 qgis/2.14.8-py2.7
```

- **proj.4** is required for coordinate transformations used later in this workbook.
- **gdal** can be loaded directly by QGIS, but visualisations used in this exercise work better if we load gdal independently
- **QGIS** is our main tool for this worksheet

```
qgis &
```

This will start QGIS. You can safely ignore warnings in this terminal window - minimise it.

Just briefly we'll introduce some terms referred to often:

![Some QGIS terms](./qgis.images/1-qgis.window.png "Some QGIS terms")


## <a name = "loadraster"></a>5. Import elevation and tree cover layers into QGIS

Follow the screenshot directions here to import your raster DEM into QGIS:


**If you are asked to select a coordinate reference system, use WGS84 / EPSG:4326**

![Import WCS image to QGIS](./qgis.images/2-load.raster.png "Find and load WCS raster")

![Show WCS layer in QGIS](./qgis.images/3-view.raster.png "Show elevation map")



### Repeat this step with your tree cover NetCDF file - load it into QGIS as a raster layer

**If you are asked to select a coordinate reference system, use WGS84 / EPSG:4326**

## <a name = "rasterstyle"></a>6. Style the tree cover layer

So far we have a bunch of grey things - let's make them colourful!

We don't need to worry about the elevation data, but we do need a useful colour scheme for vegetation data and our cadastral data. Double click your vegetation layer **in the layers panel** to bring up it's properties dialogue. From there:

1. Click 'style' in the left panel (dark background)
2. In the 'render type' dropdown, choose 'Singleband pseudocolor'
3. Leave interpolation as 'linear', and pick a colour palette. Because the data are continuous, a single hue continuous colour palette makes sense. Because it's vegetation, green also makes sense. Choose what makes sense to you.
4. Choose 'equal interval' in the 'Mode' dropdown *(try others, see what happens)*
5. Click 'classify' to show the palette in the big window, and apply with with 'apply'.
6. Click 'close' to return to your map.

![Style vegetation layer](./qgis.images/4-style.trees.png "Style vegetation layer")


<div class="alert alert-success">
<h3>Extension:</h3>
<p>
Steaming ahead? Add a graticule to your QGIS map layer...</p>
<p>
How could you automagically save WCS data with sane, memorable names?</p>

</div>

<div class="alert alert-info">
<h3>Q & A</h3>
<p>
<b>Why not just use the  QGIS OWS browser to grab these data?</b></p>
<p>
<ol>
<li>The QGIS WCS browser and THREDDS don't play well - QGIS attenuates the full THREDDS WCS URL, so it is far more convenient to form a request independently of QGIS.</li>
<li>OWS layers can't be used for processing - ie band math, and the visualisation tools we're about to use.</li>
<li>QGIS's OWS engine would attempt to load the full dataset - which is far more than we need.</li>
</ol>
<p>
If you're keen, do some exploring - we don't know *everything* about QGIS - surprise us! More about QGIS OWS capabilities can be found here: http://docs.qgis.org/2.14/en/docs/user_manual/working_with_ogc/index.html</p>
<p>
<b>Why not get these data from the filesystem?</b></p>
<p>
Great question! The main reason is that we're demonstrating QGIS as a way to fuse data from manifold sources. Over today and tomorrow you'll see a log examples of how to collect data from the filesystem in manageable chunks - which you could then analyse/visualise using the methods shown here. Demonstrating web coverage services on the VDI shows how you can pull data from many external sources to help interpret your work.</p>
</div>

<div class="alert alert-info">
<p>
For more on working with raster data, see: http://docs.qgis.org/2.14/en/docs/user_manual/working_with_raster/index.html<p>
</div>

## <a name="ingestvector"></a>4. Find a cadastral data service and import it into QGIS


### a. Finding and importing data as a service

ACT publish a lot of spatial data on their ACTMAPi interface. To shortcut, here are the cadastral section data:

http://actmapi.actgov.opendata.arcgis.com/datasets/eedcda7873934e789093b093521b0299_3

You can download a shapefile and import it to QGIS, but let's show a feature you might like to use: adding vector data as a service. At ACTMAPi, click the 'API' menu box, and copy the URL out of the GeoJSON tetxtbox:

http://actmapi.actgov.opendata.arcgis.com/datasets/eedcda7873934e789093b093521b0299_3.geojson

In QGIS, Click 'add new vector layer', select  the 'protocol' radio button, and paste the URL in. Click OK, and wait while QGIS takes a few moments to gather and render the layer.

**If you are asked to select a coordinate reference system, use WGS84 / EPSG:4326**

![add vector layer from web service](./qgis.images/5-load.geoJSON.png "Add vector layer from web service")

### b. Making the data useful - reload as a local vector layer

The layer will load - but like WCS raster data, it can't be used for analysis. Save it as a Geopackage (or shapefile) and reload the layer. Again, your ```qgis_vis_files``` directory is a good place.

Once this has been done, remove the GeoJSON layer (right click it's name in the layer panel, choose 'remove')

![Save JSON layer as geopackage](./qgis.images/6-save.vector.png "Save JSON layer as geopackage")

<div class="alert alert-success">
<p>
Why would we use a Geopackage? Why not a shapefile?<br />
<i>Hint: http://www.geopackage.org</i></p>
</div>

<div class="alert alert-info">
<p>
For more on working with vector data, see: http://docs.qgis.org/2.14/en/docs/user_manual/working_with_vector/index.html<p>
</div>

## <a name="plugins"></a>5. Install two useful QGIS plugins

To procees we need to install two QGIS plugins - **Qgis2threejs** and **Zonal statistics** :

1. Head to the **plugins** menu and choose **manage and install plugins**
2. In the resulting dialogue box, search for **Qgis2threejs**
3. Select the **Qgis2threejs** plugin and click **install**.

The **Zonal Statistics** plugin is installed by default but needs activating. Search fit **Zonal statistics** in the plugin manager, and check the box next to the plugin name. Then click **close** and you're done.

## <a name="zonalstats"></a>6. Using zonal statistics for basic analysis

We are aiming to show:

* Standard deviaton of elevation of blocks as a proxy for hilliness; and
* The mean treecoveer of each section.

The **Zonal statistics** plugin collects data from raster layers using polygons in a vector layer, and computes basic statistics for the chunk of the raster layer inside each polygon.

### a. Section hilliness

As a proxy for section hilliness, we'll use the standard deviation of elevation in each section polygon.

Head to **raster -> zonal stats** to open the plugin.

In the zonal statistics plugin dialogue, choose the DEM as the raster layer, and use band 1. Then choose your ACT blocks vector layer. In the statistics to calculate, pick an appropriate set - but include *standard deviation* - this is our roughness proxy. Add a meaningful prefix to the statistics (e.g. 'dem_'), so you can find them when you need to use them.

![HIllines using zonal statistics](./qgis.images/7-zonal.stats.png "Zonal stats example")

QGIS will spend some time calculating stats for each block, and add the output to the vector layer (act_sections) as another attribute column.


### b. Tree cover per section

For tree cover, each grid cell/pixel shows an estimate of tree cover as a percentage of the pixel. As a quick measure we could take the mean of all the pixel values inside a section to get an idea of it's tree coverage.

Open the zonal statistics plugin again, but choose the tree cover as the raster layer and use band 1. Then choose your ACT sections vector layer. In the statistics to calculate, the values we need are preselected - we'll use the mean value for each section. Again, use a meaningful prefix (for example, 'tree_') for the attribute you are adding to the act_sections dataset.

### c. Use mean tree cover per section to style our vector layer

We avoided styling our vector layer earlier, but now it's time - since we want to visualise the tree cover in each section.

Double click your act_sections layer to open it's properties dialogue, and head to the style tab. Here, we want to apply a good looking colour scale based on mean tree cover (the data we just generated). Set up as follows, so that your style dialogue looks like the screenshot below:

- Choose 'graduated' from the menu at top left
- In 'column' choose 'tree_mean' (assuming you used tree_ as a prefix in the last step)
- Choose a colour ramp
- Click 'classify'
- Click 'Apply' to preview or 'OK' to commit your changes

*Feel free to tinker with other settings - you can always remove and reload the layer to start over*

![Styling vector sections](./qgis.images/8-style.vector.png "Section styling example")

After you've clicked OK, right click your ```act_sections``` layer and select **zoom to layer** to take a closer look at the results.

![Styling vector sections](./qgis.images/9-tree.view.png "Section styling result")

<div class="alert alert-info">
<h3>We already have a result!</h3>
<p>
So far we can visualise the tree cover of sections in the ACT - but how can we relate that quickly and easily to section hillines? And how could we interactively explore the relationship?</p>
</div>

<div class="alert alert-success">
<p>
<b>Question:</b><br />
Is a graphical GIS the only way to collect zonal statistics using a vector layer to segment raster data?</p>
</div>

## <a name="visualise"></a>7. Using a simple 3D visualisation to complete the picture

So far we've imported three different datasets into QGIS and created some new attributes on our vector dataset. How 
* classify and colour blocks by vegetation cover
* visualise block hilliness as the height of an extruded column

In this scheme, if hillier blocks are more vegetated, dark green blocks will visualise as taller columns. If hiller blocks are less vegetated, light green blocks will visualise as taller columns. Lets test it out!

Here we use the second plugin - **Qgis2threejs**. This renders the current screen to a WebGL map in a .html page using three.js - with some neat data visualisation features. You can open the result as an interactive map in a web browser.

### a. Setting up - coordinate transformation in QGIS

So far we've worked in a WGS84 (EPSG:4326) coordinate system - but in order to render our map in three.js, we need to project our data into something which has units of metres, not degrees. Let's choose GDA94/MGA55 (EPSG:28355):

1. Locate the panel at the lower right of the QGIS window which says 'EPSG:4326'
2. Click it to open a CRS selection dialog
3. Select 'Enable 'on the fly' CRS transformation (OTF)' at the top left
4. Enter 'MGA 55' in the 'filter' box, then highlight GDA94 /MGA 55 in the 'coordinate systems of the world...' box. It should show up in the 'selected CRS' panel.
5. Click OK.

![Set OTF CRS](./qgis.images/10-crs.otf.png "Set OTF transformation")

You'll see that everything has warped a touch, and your CRS panel (lower right) reflects your choice. Proj.4 handles all the rest for you!

<br />
<div class="alert alert-info">
<p>
<b>Transformations made simple!</b><br />
QGIS sits on top of the Proj.4 library - meaning that any of the transformation capabilities offered by Proj.4 are available. Read more here: http://docs.qgis.org/2.14/en/docs/user_manual/working_with_projections/working_with_projections.html</p>
</div>

### b. Set a clipping frame

Qgis2threejs attempts to render the whole map window. We want to limit or map to the extents of our DEM - so we can either zoom the map window in so that our region of interest occupies the whole window, or set a clipping polygon in a new vector layer. Here, we zoom in so that our region of interest fills the map window.

The easiest way to achieve this is zooming to the extents of the ACT sections layer you've made. Right click the layer name, and select **zoom to layer**. You may need to use the **hand** (from the upper row of icons in your QGIS window) to move your map around so that your map window is completely filled, and the **magnifying glass** or **mouse scroll wheel** to adjust the zoom level. You should end with something like the second screenshot in section 10 (above).

<br />
<div class="alert alert-info">
<p>
The next steps render the entire map window in a WebGL-based 3D viewer - it's OK to leave some empty space around your region of interest, but it works better to completely fill your map window with relevant data.</p>
</div>

### c. Setting up in Qgis2threejs

Head to **web -> Qgis2threejs** to open the plugin dialogue. Click 'world' in the left pane, here we define basic parameters about the map we're creating. It's usually prettier to apply some vertical exaggeration in Australia, try between 1.5 and 5. Using 5 gives Canberra a pretty alpine feel.

![QGIS2threejs setup](./qgis.images/11-qgis2threejs.world.png "QGIS2threejs setup - world")

Next click 'DEM'. Here you select the SRTM data you grabbed via WCS as the dem to build terrain. You can set an image to be draped on the DEM - map window view, a specific layer, an image or a plain colour. This example uses the DEM layer to colour the terrain map. Also turn shading on to get pretty hillshading.

![QGIS2threejs setup](./qgis.images/12-qgis2threejs.dem.png "QGIS2threejs setup - DEM")

Finally check the radio button next to our ACT sections layer in the left panel. Here we set up some visualisation parameters for our vector layer (which contains the section-level treecover statistics we generated). Set the Z coordinate to'Absolute value', and enter 580 m. We're going to compare the heights of extruded polygons, so we should start them all in the same place!

Keep the colour as 'feature style', set 'Transparency' to 20-30 and finally, choose a data source to detemine extruded polygon heights. We finally relate the hilliness of sections to tree cover here - choose **dem_stdev**, and a multiplier (10 works well in this example).

![QGIS2threejs setup](./qgis.images/13-qgis2threejs.polygon.png "QGIS2threejs setup - DEM")

### d. show the map!

We finally use our ```qgis_vis_map``` directory - browse to it at the 'output html file path' box and use a memorable name to save your output html file. Then click **run**. All being well firefox will open and display an interactive map like the one below! See section 13 for links to prebuilt versions hosted at github.io.

**If the map doesn't pop up right away** navigate to the place you just stored the .html file, and open it in a browser.

**If firefox complains about being unresponsive, tell it to wait. If if asks about an unresponsive script, tell it to continue**

**If there are a lot of users on the VDI, exploring your map might be a little slow**

![End result](./qgis.images/0-3d.map.png "End result of this session")

## <a name="interpret"></a>12. So what do our results mean? Interpreting our picture

If hilly blocks have generally more tree cover than flatter blocks, short columns should be lighter shades of blue (in this colour scheme).

...but that's not necesarily what we see! Why not?


<div class="alert alert-success">
<h3>Extension activities</h3>

<ol>
<li>Which suburbs have the most trees? which is the hilliest? * **hint** - use another QGIS plugin, and another web mapping data source - and see the example in section 13*
<li>Create a totally new interactive 3D model using the stack we've just been working with, and give us a URL to see your work!</li>
<li>Visualise the same result a different way - do we *really* need qgis2threejs?</li>
</ol>
</div>

##  <a name="share"></a>13. Sharing the map

The map you just created using QGIS2threejs is a fully stand-alone webGL application that can be dropped into any web server to share with collaborators. 

You can't serve data directly from the VDI - but you can copy the whole ```qgis_vis_map``` folder to some web host (e.g. github.io) and share with the world. Here's an example:

https://adamsteer.github.io/nci_samples/qgis2threejs/treecover.html

Here's another example with an OpenStreetMap layer rendered on the DEM:

https://adamsteer.github.io/nci_samples/qgis2threejs/treecover-osm.html

What are some other ways to share QGIS projects?

<div class="alert alert-success">
<h3>Possible solutions to questions</h3>

<h4>Sane WCS coverage request names</h4>
<p>
One option is to use curl on the command line:</p>
<p>
<code>curl -o SRTM_dem_139_36.tiff 'http://dapds00.nci.org.au/thredds/wcs/rr1/Elevation/NetCDF/1secSRTM_DEMs_v1.0/DEM/Elevation_1secSRTM_DEMs_v1.0_DEM_Mosaic_dem1sv1_0.nc?service=WCS&version=1.0.0&request=GetCoverage&Coverage=elevation&bbox=148.7,-35.8,149.5,-35.1&format=GeoTIFF_Float'</code>
</p>
<p>
...are there others? What was yours?</p>

<h4>Zonal statistics right in a notebook</h4>
<p>
You could also try rasterstats: https://github.com/perrygeo/python-rasterstats - any other suggestions?</p>

<h4>Other ways to share your QGIS projects, and other 3D visualisers</h4>
<p>
For simple projects with CCBY4 data, the QGIS cloud is one way of sharing your results. Another is a Jupyter notebook hosted on github as a gist, a notebook, or as a github.io page. What was your approach?</p>
</div>

<div class="alert alert-warning">
<h3>Further reading</h3>
<p>
Here is a great tutorial on netCDF and QGIS: http://www.ggiuliani.ch/download/netcdf_qgis_GG.pdf - covering a lot more than we did here, and useful for working with data directly on the VDI filesystem.</p>
<p>
Also, inspect the result of your work - what is Three.js doing? What can you apply from this example to other work? The Qgis2threejs plugin can only add so much - how could you render additional datasets in the same map?
</div>

## Summary

Here we:
- Used a graphical GIS available on the VDI
- Used OGC web services available at NCI
- Demonstrated pulling data from external web services
- Demonstrated shaping data in the filesystem for merging with web service data
- Used these data to perform a basic analysis
- Learned one method of visualising and sharing results

# Thanks! Discussion and suggestions welcome.